In [ ]:
from spacerocks import SpaceRock
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy.time import Time
import astropy.units as u
import numpy as np
from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u
import easyaccess as ea
import matplotlib.pyplot as plt
from DEEP_pointings import DEEP_fields_2019, DEEP_fields_2020
from compute_chip import compute_chip
from matplotlib.patches import Ellipse

In [ ]:
from astroquery.jplhorizons import Horizons

# Identify known Jupiter Trojans in DES Y6A1 single exposure
To identify JTs in DES data, we first obtain single exposures in DES Y6A1 that have known JTs within the field of view. The first step is to read in data from MPC of minor planets and save them as csv file 

In [ ]:
dfo = pd.read_json('https://minorplanetcenter.net/Extended_Files/mpcorb_extended.json.gz') 
dfo.to_csv('mpcorb_extended.csv', index=None)
dfo = pd.read_csv('mpcorb_extended.csv') 

This is the code for getting field of view of DECam and estimating whether the object is inside that field.

In [ ]:
class DECamField(object):
    def __init__(self, pos):
        self.ra = pos.ra.deg
        self.dec = pos.dec.deg

    def ellipse(self):
        # An approximation to the DECam field of view, suitable e.g. for plotting
        semimajor_deg = 1.08
        semiminor_deg = 0.98
        center = (self.ra, self.dec)
        rotation = 0
        return Ellipse(center, 2*semimajor_deg, 2*semiminor_deg, rotation, fill=False, ec='k')
    
    def contains(self, ra1, dec1):
        # returns True if the point (ra1, dec1) lies inside the field
        radiff = ra1-self.ra
        if radiff > 360: radiff -= 2*360
        return radiff**2/1.08**2 + (dec1-self.dec)**2/0.98**2 <= 1.0

This code read in date, exposure number, and pointings of the DECam. It caculate the ephermis motions of known Jupiter Trojans (JTs) in MPC in the date of interest. Then, it returns the known JTs positions if they are within the field of DECam at the time.

In [ ]:
def give_ccd(date,expnum,pointings):
    df = dfo.loc[dfo.Orbit_type=="Jupiter Trojan"] #Known Jupiter Trojans in MPC
    rocks = SpaceRock(a=df.a.values, 
                  e=df.e.values, 
                  inc=df.i.values, 
                  arg=df.Peri.values, 
                  node=df.Node.values, 
                  t_peri=df.Tp.values, 
                  epoch=df.Epoch.values,
                  H=df.H.values, 
                  name=df.Principal_desig.values.astype(str),
                  precise=False,
                  coordinates='keplerian',
                  angles='degrees',
                  frame='heliocentric',
                  obscode='W84')
    dates = Time(date, format='iso', scale='utc')
    prop1 = rocks.propagate(dates.jd, model=1) #Use space rock to derive the ephermis motions of known JTs.
    obs1o = prop1.observe('W84')
    df['ra'] =obs1o.ra.deg 
    df['dec'] =obs1o.dec.deg 
    df['expnum']=expnum
    ccd = []
    for ind, row in df.iterrows(): #Estimate whether the ephermis positions of knwon JTs are within the field of view of DECam.
            target_pos = SkyCoord(row['ra'], row['dec'], frame='icrs', unit=(u.deg, u.deg))
            ccdnam, ccdnum = compute_chip(target_pos.ra.radian, target_pos.dec.radian, pointings.ra.radian, pointings.dec.radian)
            ccd.append(ccdnum)
    df['ccd'] = ccd
    df2 = df.loc[df.ccd>0] #Get the known JTs of those in the field
    return df2

Now, we get date, pointing and exposure number from DES Y6A1 data. The format are changed to make sure they can be read.

In [ ]:
pointing = pd.read_csv('DES pointing/DES_Y6A1_exposures.csv')
pointgra = pointing['RADEG'].values.tolist()
pointgdec = pointing['DECDEG'].values.tolist()
point = []
for i,j in zip(pointgra,pointgdec):
    pointinge = SkyCoord(i, j, frame='icrs', unit=(u.deg, u.deg))
    point.append(pointinge)

In [ ]:
pointing = pd.read_csv('DES pointing/DES_Y6A1_exposures.csv')
time = pointing['DATE_OBS']
d = []
for i in time:
    a = i.replace("T"," ")
    c = a.replace("b","")
    e = eval(c)
    d.append(e)
time2 = d

In [ ]:
pointing = pd.read_csv('DES pointing/DES_Y6A1_exposures.csv')
expnum = pointing['EXPNUM'].values.tolist()

In [ ]:
desinfo = pd.DataFrame() #The pointings, exposure number, and data of DES Y6A1 single exposure.
desinfo['telpos'] = point
desinfo['expnum'] = expnum
desinfo['date'] = time2

Finally, we run all the DES Y6A1 exposure to search for known JTs. The search is confined to ecliptic latitude from -30◦to 30◦. This requires a long time to run, so multiple notebooks were created and ran at once in moriarty. 

In [ ]:
dd = pd.DataFrame()
for i in desinfolp: #The desinfo is run in multiple notebooks to make it faster.
    dateten = i[2]
    expnumten = i[1]
    poten = i[0]
    eclipticlat = poten.geocentrictrueecliptic.lat.deg
    df2 = pd.DataFrame()
    if eclipticlat < 30 and eclipticlat > -30:
        df2 = give_ccd(date = dateten,expnum = expnumten,pointings= poten) 
        DESJTten = df2[['expnum','Principal_desig','ra', 'dec','ccd']]
        dd = pd.concat([dd,DESJTten])
    else: 
        pass

# Obtaining DES photometry
Here we match identified JTs with Y6A1_FINALCUT_OBJECT objects to get photometry in DES. The object within 2'' of Y6A1_FINALCUT_OBJECT are selected. We excluded objects within 1'' that are in Y6A2_COADD_OBJECT_SUMMARY table, because they are likely to be stationary objects and are not of interest for your purpose. 

The Nepoch value is also extracted from Y6A2_COADD_OBJECT_SUMMARY table for objects within 1''. This value means the number of single exposure used for a COADD objects, and we find that if Nepoch value = 1, then the COADD object is likely to be a JT. Thus, coadd objects with Nepoch value smaller or equal to 1 but not all of them are zero are kept.

At this stage, we obtained 12057 exposures corresponding to 888 unique objects

In [ ]:
def se_query(expnum,name,ra, dec, db=db):
    one_arcsec = 1/3600.
    query = f"select y.ra, y.dec, y.band ,y.flux_auto, y.fluxerr_auto, y.expnum, y.nite, y.fwhm_world from Y6A1_FINALCUT_OBJECT y \
    where y.expnum={expnum} and y.ra between {ra}-2*{one_arcsec} and {ra}+2*{one_arcsec} \
    and y.dec between {dec}-2*{one_arcsec} and {dec}+2*{one_arcsec}"
    result = db.query_to_pandas(query) 
    result['name'] = name
    return result

def coadd_query(ra, dec, db=db):
    one_arcsec = 1/3600.
    query = f"select c.ra, c.dec, c.flux_auto_g,c.flux_auto_i,c.flux_auto_r,c.flux_auto_y,\
    c.flux_auto_z,c.Nepochs_g,c.Nepochs_r,c.Nepochs_i,c.Nepochs_y,c.Nepochs_z from Y6A2_COADD_OBJECT_SUMMARY c \
    where c.ra between {ra}-{one_arcsec} and {ra}+{one_arcsec} and c.dec between {dec}-{one_arcsec} and {dec}+{one_arcsec} \
    and (c.Nepochs_g>1 or c.Nepochs_r>1 or c.Nepochs_i>1 or c.Nepochs_y>1 or c.Nepochs_z>1 or (c.Nepochs_g=0 and c.Nepochs_r=0 and c.Nepochs_i=0 and c.Nepochs_y=0 and c.Nepochs_z=0))"
    result = db.query_to_pandas(query)
    return result

sejt = pd.DataFrame()
for pred in desjt: #a tuple with expnum and position.
    expnum = pred[0]
    name = pred[1]
    ra = float(pred[2])
    dec = float(pred[3])
    df_se = se_query(expnum, name,ra, dec, db=db)
    df_coadd = coadd_query(ra, dec, db=db)
    df_se.to_csv()
    if df_coadd.empty == True:
        sejt = pd.concat([sejt,df_se])
    else:
        pass

In [ ]:
def Nepoch_query(ra, dec, db=db):
    one_arcsec = 1/3600.
    query = f"select c.Nepochs_g,c.Nepochs_r,c.Nepochs_i,c.Nepochs_y,c.Nepochs_z, c.dec, c.flux_auto_g,c.flux_auto_i,c.flux_auto_r,c.flux_auto_y,c.flux_auto_z from Y6A2_COADD_OBJECT_SUMMARY c \
    where c.ra between {ra}-{one_arcsec} and {ra}+{one_arcsec} and c.dec between {dec}-{one_arcsec} and {dec}+{one_arcsec}"
    result = db.query_to_pandas(query)
    return result

### Positions uncertainties of JTs
Now we estimate positions uncertainties of JTs ephermis positions, and only choose those with < 2'' uncertainties in ra and dec. After contraining all the identified JTs to this criterion, we arrived at a number of  860 unique Jupiter Trojans with 11603 number of single exposures

The time range is set to be within 1 second of the date of observation; as the JPL horizons needs to take a range of time for the estimation of uncertainties.

In [ ]:
desjt = pd.read_csv('DES_Trojans_mpc(2).csv') #File of JTs 

In [ ]:
a = desjt.values.tolist()
delta_ra = []
delta_dec = []
for i in a:
    name = i[1] #name of JTs
    i = i[11] #Time of the JTs
    d = i.replace("T"," ")
    c = d.replace("b","")
    e = eval(c)
    date = Time(e, format='iso', scale='utc') #Time of observaiton of JTs.
    stopttime = starttime + 1*u.second 
    s = str(stopttime)
    obj = Horizons(id=name, location='W84', #Calculate ephermis motions in JPL
                epochs={'start':str(starttime), 'stop':str(s),
                       'step':'1m'})
    eph = obj.ephemerides()
    delta_ra.append(eph['RA_3sigma']) #Store the uncertainties of JTs eperhmis motions from JPL
    delta_dec.append(eph['DEC_3sigma'])
delta_ra = np.array(delta_ra)
delta_dec = np.array(delta_dec)
delta_ra.ravel()
delta_dec.ravel()

### Further constraints on the selected Trojans and photometry

Here are the code to get zero point magniutde, apparent magniutde, and absolute magniutde. To further improve the quality of photometry, we require that the number of exposure of each Trojans to be bigger than 1 and magnitude error to be smaller than 0.1. 

After this, we obtain 269, 365, 307, and 237 L5 Trojans and 14,  13,  17,  and 12 L4 Trojans with r, g, i, and z band measurement respectively. Then the color color diagram, color magniutde, and color distributions can thus be analyzed for JTs.


### zero point magniutde

In [ ]:
def zero_mag(expnum,ccd,band,db=db):
    query = f"select z.expnum,z.mag_zero,z.SIGMA_MAG_ZERO,z.band,z.ccdnum from Y6A1_ZEROPOINT z \
    where z.expnum={expnum} and z.band='{band}'and z.ccdnum={ccd}"
    result = db.query_to_pandas(query) 
    return result

def zero_mag2(expnum,ccd,band,db=db):
    query = f"select z.expnum,z.mag_zero,z.SOURCE,z.VERSION,z.SIGMA_MAG_ZERO,z.band,z.ccdnum from Y3A2_ZEROPOINT z \
    where z.expnum={expnum} and z.band='{band}'and z.ccdnum={ccd} and z.VERSION = 'v2.0' and  z.source='FGCM'"
    result = db.query_to_pandas(query) 
    return result

def zero_mag3(expnum,ccd,band,db=db):
    query = f"select z.expnum,z.mag_zero,z.SOURCE,z.VERSION,z.SIGMA_MAG_ZERO,z.band,z.ccdnum from Y3A2_ZEROPOINT z \
    where z.expnum={expnum} and z.band='{band}'and z.ccdnum={ccd} and z.VERSION = 'QUAT_2A' and  z.source='PGCM_FORCED'"
    result = db.query_to_pandas(query) 
    return result

Obtain zero point magniutde from Y6A1_ZEROPOINT and Y3A2_ZEROPOINT. The zero point magnitudes are added to the original file of JTs and u and Y band measurements are not ocnsidered. Still missing 207 zero poing magniutde, and most of them are likely to be bad values, e.g. -9999.

In [ ]:
desco = pd.read_csv('DES_JT_final_H&P.csv')
desco = desco.loc[(desco['BAND']!='u')&(desco['BAND']!='Y')]
noexp = pd.read_csv('DES_mis_m0.csv')
noexp = flat(noexp.values.tolist())
expn = pd.DataFrame()
expnume = pd.DataFrame()
expnume2 = pd.DataFrame()
expnume3 = pd.DataFrame()
exx = pd.DataFrame()
a = 0
rae= []
dece= []
bande= []
expe= []
namee= []
ccde= []
fluxe= []
flux_errore= []
nitee= []
fhwme= []
Lne= []
missexp = []
Hmage= []
datee= []
what=[]
for i in desco.values.tolist():
    exp = i[5];band = i[2];ccd = i[12];ra = i[0];dec = i[1];flux = i[3];flux_error= i[4];nite= i[6];fhwm= i[7];name = i[8];Ln = i[11];date = i[10];Hmag = i[9]
    expnume = zero_mag(exp,ccd,band,db=db) #Primary
    expnume2 = zero_mag2(exp,ccd,band,db=db) #secondary primary
    expnume3 = zero_mag3(exp,ccd,band,db=db) #secondary seconary
    if ((expnume['MAG_ZERO'].values<0)or (expnume.empty ==True)) and((expnume3['MAG_ZERO'].values<0)or (expnume3.empty ==True))and ((expnume2['MAG_ZERO'].values>0)):
                rae.append(ra)
                dece.append(dec)
                bande.append(band)
                expe.append(exp)
                namee.append(name)
                ccde.append(ccd)
                fluxe.append(flux)
                flux_errore.append(flux_error)
                nitee.append(nite)
                fhwme.append(fhwm)
                Lne.append(Ln)
                Hmage.append(Hmag)
                datee.append(date)
                expnume2.to_csv()
                expn = pd.concat([expn,expnume2])
    elif ((expnume['MAG_ZERO'].values<0)or (expnume.empty ==True)) and(expnume3['MAG_ZERO'].values>0) and (expnume2['MAG_ZERO'].values>0) :
                rae.append(ra)
                dece.append(dec)
                bande.append(band)
                expe.append(exp)
                namee.append(name)
                ccde.append(ccd)
                fluxe.append(flux)
                flux_errore.append(flux_error)
                nitee.append(nite)
                fhwme.append(fhwm)
                Lne.append(Ln)
                Hmage.append(Hmag)
                datee.append(date)
                expnume2.to_csv()
                expn = pd.concat([expn,expnume2])
    elif ((expnume['MAG_ZERO'].values<0)or (expnume.empty ==True)) and((expnume2['MAG_ZERO'].values<0)or (expnume2.empty ==True))and ((expnume3['MAG_ZERO'].values>0)):
                rae.append(ra)
                dece.append(dec)
                bande.append(band)
                expe.append(exp)
                namee.append(name)
                ccde.append(ccd)
                fluxe.append(flux)
                flux_errore.append(flux_error)
                nitee.append(nite)
                fhwme.append(fhwm)
                Lne.append(Ln)
                Hmage.append(Hmag)
                datee.append(date)
                expnume3.to_csv()
                expn = pd.concat([expn,expnume3])
    elif expnume['MAG_ZERO'].values>0:
                rae.append(ra)
                dece.append(dec)
                bande.append(band)
                expe.append(exp)
                namee.append(name)
                ccde.append(ccd)
                fluxe.append(flux)
                flux_errore.append(flux_error)
                nitee.append(nite)
                fhwme.append(fhwm)
                Lne.append(Ln)
                Hmage.append(Hmag)
                datee.append(date)
                expnume.to_csv()
                expn = pd.concat([expn,expnume])
    else: #Negative in Y6A1 and no valiue in Y3A2
                what.append(exp)

In [ ]:
Store the data

In [ ]:
expn['ra'] = rae
expn['dec'] = dece
expn['band'] =bande
expn['expnum'] =expe
expn['name'] =namee
expn['ccd'] =ccde
expn['FLUX_AUTO'] =fluxe
expn['FLUXERR_AUTO'] =flux_errore
expn['NITE'] =nitee
expn['FWHM_WORLD'] =fhwme
expn['Ln'] =Lne
expn['Date'] =datee
expn['H'] =Hmage
expn.to_csv('DES_JT_final_m0.csv', index=None)

### Absolute magnitude
Then, I derive aboslute magniutde using space rock. First, the apparent magniutde is obatined from flux and zero point magniutd following m=m0 + log10(flux). 

Then, I match the name of each JTs with that of MPC to get their orbital parameters. The date and apparent magniutde are from our catalog and other parameters are from MPC.

In [ ]:
desco = pd.read_csv('DES_JT_final_m0.csv')
for i in desco.values.tolist():
    mag.append(-2.5*np.log10(i[13])+i[1])
desco = pd.read_csv('DES_JT_final_m0.csv')
desco['Apparent_Mag'] = mag

In [ ]:
d = []
Hg = []
desco = pd.read_csv('DES_JT_final_m0.csv')
for i in desco.values.tolist():
    date  = i[18]
    magg = i[20]
    name = i[11]
    a = date.replace("T"," ")
    df = dfo.loc[dfo.Principal_desig==name]
    rock = SpaceRock(a=df.a.values, 
                  e=df.e.values, 
                  inc=df.i.values, 
                  arg=df.Peri.values, 
                  node=df.Node.values, 
                  t_peri=df.Tp.values, 
                  epoch=a,
                  name=df.Principal_desig.values.astype(str),
                  mag= magg,
                  precise=False,
                  input_coordinates='keplerian',
                  input_frame='heliocentric',
                  input_angles='degrees',
                  obscode='W84')
    H = rock.calc_H(obscode='W84')
    H = H[0]
    H.ravel()
    Hg.append(H)

Now we get absolute magniutde and its error for each JTs by taking the average over all of the exposures. 
Apparent magniutde is also derived in case needed.

In [ ]:
def ABmager(name,band,L):
    desco = pd.read_csv('DES_JT_final_m0.csv')
    desco = desco.values.tolist()
    mager = []
    mag = []
    name = name
    band = band
    L = L
    for i in desco:
        if (name == i[11]) &(band==i[3])&(L==i[17]): #Match with one JT.
            mager.append(i[14]**2) #FLux error
            mag.append(i[13]) #FLux
    magf = np.average(mag) 
    magfer = np.sqrt(np.sum(mager))/np.sqrt(len(mager))  
    if len(mag)>1: #constraint that number of exposure > 1
        magff =(-2.5/np.log(10))*(magfer/magf)  #Magnitude error
    else:
        magff = 0
    return magff

In [ ]:
def Hmag(name,band,L):
    desco = pd.read_csv('DES_JT_final_m0.csv')
    desco = desco.values.tolist()
    mag = []
    name = name
    band = band
    L = L
    for i in desco:
        if (name == i[11]) &(band==i[3])&(L==i[17]): 
            hh = i[20].replace("[","")
            hh2 = hh.replace("]","")
            mag.append(i[21])
    if len(mag)>1: #With exposure more than 1 Nepoch requirement > 1
        magff = np.average(mag)
    else:
        magff = 0
    return magff

In [ ]:
def Apmag(name,band,L): #apparent mag
    desco = pd.read_csv('DES_JT_final_m0.csv')
    desco = desco.values.tolist()
    mag = []
    name = name
    band = band
    L = L
    for i in desco:
        if (name == i[11]) &(band==i[3])&(L==i[17])&(i[1]>0):
            mag.append(np.array([i[20]]).astype(np.float))
    if len(mag)>1: #With exposure more than 1
        magff = np.average(mag)
    else:
        magff = 0
    return magff

Now store name, absolute magnitude, magniutde error, and apparent magniutde into a Panda datafram so they can plotted easily. This is for each band (g, r, i, z) and each cloud (L4 and L5).

In [ ]:
gmag = pd.DataFrame()
gg =  desco[(desco['BAND']=='g')&(desco['Ln']=='L5')]['name'].unique()
JTmag_g_L5 = pd.DataFrame()
ggmag=[]
gpmag=[]
ggmager=[]
for i in gg:
    mag = Hmag(name = i,band = 'g',L = 'L5')
    ggmag.append(mag)
    maga = Apmag(name = i,band = 'g',L = 'L5')
    gpmag.append(maga)
    mager = ABmager(name = i,band = 'g',L = 'L5')
    ggmager.append(mager)
JTmag_g_L5['name'] = gg
JTmag_g_L5['magH_g'] = ggmag
JTmag_g_L5['mag_ger'] = ggmager
JTmag_g_L5['mag_g'] = gpmag

In [ ]:
gmag = pd.DataFrame()
gg =  desco[(desco['BAND']=='g')&(desco['Ln']=='L4')]['name'].unique()
JTmag_g_L4 = pd.DataFrame()
ggmag=[]
gpmag=[]
ggmager=[]
for i in gg:
    mag = Hmag(name = i,band = 'g',L = 'L4')
    ggmag.append(mag)
    maga = Apmag(name = i,band = 'g',L = 'L4')
    gpmag.append(maga)
    mager = ABmager(name = i,band = 'g',L = 'L4')
    ggmager.append(mager)
JTmag_g_L4['name'] = gg
JTmag_g_L4['magH_g'] = ggmag
JTmag_g_L4['mag_ger'] = ggmager
JTmag_g_L4['mag_g'] = gpmag

In [ ]:
rr =  desco[(desco['BAND']=='r')&(desco['Ln']=='L5')]['name'].unique()
JTmag_r_L5 = pd.DataFrame()
rmag=[]
rpmag=[]
rmager=[]
for i in rr:
    mag = Hmag(name = i,band = 'r',L = 'L5')
    rmag.append(mag)
    maga = Apmag(name = i,band = 'r',L = 'L5')
    rpmag.append(maga)
    mager = ABmager(name = i,band = 'r',L = 'L5')
    rmager.append(mager)
JTmag_r_L5['name'] = rr
JTmag_r_L5['magH_r'] = rmag
JTmag_r_L5['mag_rer'] = rmager
JTmag_r_L5['mag_r'] = rpmag

In [ ]:
rr =  desco[(desco['BAND']=='r')&(desco['Ln']=='L4')]['name'].unique()
JTmag_r_L4 = pd.DataFrame()
rmag=[]
rpmag=[]
rmager=[]
for i in rr:
    mag = Hmag(name = i,band = 'r',L = 'L4')
    rmag.append(mag)
    maga = Apmag(name = i,band = 'r',L = 'L4')
    rpmag.append(maga)
    mager = ABmager(name = i,band = 'r',L = 'L4')
    rmager.append(mager)
JTmag_r_L4['name'] = rr
JTmag_r_L4['magH_r'] = rmag
JTmag_r_L4['mag_rer'] = rmager
JTmag_r_L4['mag_r'] = rpmag

In [ ]:
rr =  desco[(desco['BAND']=='i')&(desco['Ln']=='L5')]['name'].unique()
JTmag_i_L5 = pd.DataFrame()
imag=[]
ipmag=[]
imager=[]
for i in rr:
    mag = Hmag(name = i,band = 'i',L = 'L5')
    imag.append(mag)
    maga = Apmag(name = i,band = 'i',L = 'L5')
    ipmag.append(maga)
    mager = ABmager(name = i,band = 'i',L = 'L5')
    imager.append(mager)
JTmag_i_L5['name'] = ii
JTmag_i_L5['magH_i'] = imag
JTmag_i_L5['mag_ier'] = imager
JTmag_i_L5['mag_i'] = ipmag

In [ ]:
ii =  desco[(desco['BAND']=='i')&(desco['Ln']=='L4')]['name'].unique()
JTmag_i_L4 = pd.DataFrame()
imag=[]
ipmag=[]
imager=[]
for i in rr:
    mag = Hmag(name = i,band = 'i',L = 'L4')
    imag.append(mag)
    maga = Apmag(name = i,band = 'i',L = 'L4')
    ipmag.append(maga)
    mager = ABmager(name = i,band = 'i',L = 'L4')
    imager.append(mager)
JTmag_i_L4['name'] = ii
JTmag_i_L4['magH_i'] = imag
JTmag_i_L4['mag_ier'] = imager
JTmag_i_L4['mag_i'] = ipmag

In [ ]:
zz =  desco[(desco['BAND']=='z')&(desco['Ln']=='L5')]['name'].unique()
JTmag_z_L5 = pd.DataFrame()
zmager=[]
zpmag=[]
zmag=[]
for i in rr:
    mag = Hmag(name = i,band = 'z',L = 'L5')
    zmag.append(mag)
    maga = Apmag(name = i,band = 'z',L = 'L5')
    zpmag.append(maga)
    mager = ABmager(name = i,band = 'z',L = 'L5')
    zmager.append(mager)
JTmag_z_L5['name'] = zz
JTmag_z_L5['magH_z'] = zmag
JTmag_z_L5['mag_zer'] = zmager
JTmag_z_L5['mag_z'] = zpmag

In [ ]:
zz =  desco[(desco['BAND']=='z')&(desco['Ln']=='L4')]['name'].unique()
JTmag_z_L4 = pd.DataFrame()
zmag=[]
zpmag=[]
zmager=[]
for i in zz:
    mag = Hmag(name = i,band = 'z',L = 'L4')
    zmag.append(mag)
    maga = Apmag(name = i,band = 'z',L = 'L4')
    zpmag.append(maga)
    mager = ABmager(name = i,band = 'z',L = 'L4')
    zmager.append(mager)
JTmag_z_L4['name'] = zz
JTmag_z_L4['magH_z'] = zmag
JTmag_z_L4['mag_zer'] = zmager
JTmag_z_L4['mag_z'] = zpmag

Finally, the code to plot color color diagram, and that of color magniutde diagram is similar.

In [ ]:
#color color diagram. 
riL5s = []
izL5s = []
riL5ser = []
izL5ser = []
for a in JTmag_z_L5.values.tolist():
    if (a[1]>0)and(np.fabs(a[2])<0.1):
        name = a[0]
        mag = a[1]
        mager = a[2]
        for b in JTmag_r_L5.values.tolist():
            if (b[1]>0)and(np.fabs(b[2])<0.1):
                name2 = b[0]
                mag2 = b[1]
                mag2er = b[2]
                for c in JTmag_i_L5.values.tolist():
                    if (c[1]>0)and(np.fabs(c[2])<0.1):
                        name3 = c[0]
                        mag3 = c[1]
                        mag3er = c[2]
                        if (name == name2)&(name3 == name2):
                            iz = mag3 - mag
                            ri = mag2 - mag3
                            izer = np.sqrt(mager**2+mag3er**2)
                            rier = np.sqrt(mag2er**2+mag3er**2)
                            riL5ser.append(rier)
                            izL5ser.append(izer)
                            riL5s.append(ri)
                            izL5s.append(iz)
riL4s = []
izL4s = []
riL4ser = []
izL4ser = []
for a in JTmag_z_L4.values.tolist():
    if (a[1]>0)and(np.fabs(a[2])<0.1):
        name = a[0]
        mag = a[1]
        mager = a[2]
        for b in JTmag_r_L4.values.tolist():
            if (b[1]>0)and(np.fabs(b[2])<0.1):
                name2 = b[0]
                mag2 = b[1]
                mag2er = b[2]
                for c in JTmag_i_L4.values.tolist():
                        if (c[1]>0) and(np.fabs(c[2])<0.1):
                            name3 = c[0]
                            mag3 = c[1]
                            mag3er = c[2]
                            if (name == name2)&(name3 == name2):
                                iz = mag3 - mag
                                ri = mag2 - mag3
                                izer = np.sqrt(mager**2+mag3er**2)
                                rier = np.sqrt(mag2er**2+mag3er**2)
                                riL4ser.append(rier)
                                izL4ser.append(izer)
                                riL4s.append(ri)
                                izL4s.append(iz)
    

plt.figure(figsize=(6,6),frameon=False)
ax2=plt.subplot(111)
plt.errorbar(riL5s, izL5s, xerr=riL5ser, yerr=izL5ser,  ms=5,fmt="o", color="b",lw=0.8 ,capsize=1,label='L5')
plt.errorbar(riL4s, izL4s, xerr=riL4ser, yerr=izL4ser,  ms=5,fmt="o", color="r",lw=0.8 ,capsize=1,label='L4')
plt.errorbar(0.12, 0.04, xerr=0.01, yerr=0.02,ms=10, fmt="v", color="g", capsize=2,label='Solar color')
plt.xlabel('Hr-Hi')
plt.title('ri/iz of Jupiter Trojans')
plt.ylabel('Hi-Hz')
plt.grid()
ax2.legend(prop={'size': 10})
plt.savefig('izri',bbox_inches='tight',transparent=False,dpi=400)
plt.show()